In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import pymongo
import random
from tqdm import tqdm
import os
tqdm.pandas()
from datetime import datetime
import requests
import re
import os



nlp = en_core_web_sm.load()
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [2]:
def getDeleted():
    deletedData = []
    db = db_client[ "tweets_pipeline_v2"].find({"$or": [{"status":"pipelined"}],
                                                  "extracted_text": {"$exists": True},
                                                   })
    for i in tqdm(db, total = db.count()):
        _id = str(i['_id'])
        urls = i["img_urls"]
        reason=["pipelined"]
        deletedData.append([_id, urls, reason])
    
    totalDeleted = pd.DataFrame(deletedData, columns=["ID", "url", "GutenTagTrash"])    
    totalDeleted.reset_index(inplace=True, drop=True)

    return totalDeleted

In [3]:
totalDeleted = getDeleted()
print("totalDeleted:", totalDeleted.shape[0])
totalDeleted.head(15)

100%|██████████| 2129/2129 [00:00<00:00, 6361.25it/s]

totalDeleted: 2129


,ID,url,GutenTagTrash
0,5e8eea00ecd3bcd5b762848f,https://pbs.twimg.com/media/EPnkYmxX4AMyTiv.png,[pipelined]
1,5e90a8ceba684b187ba42152,https://pbs.twimg.com/media/EQhFRt5XsAA0HYX.jpg,[pipelined]
2,5eaab66ba57627b021b6d5d0,https://pbs.twimg.com/media/ET5CeENUcAALMuD.jpg,[pipelined]
3,5eaae294684281d51dabf5d2,https://pbs.twimg.com/media/ETbE0DJXkAY3FFA.jpg,[pipelined]
4,5eab1b1b25234e95b3939073,https://pbs.twimg.com/media/EUXD1RtWoAMwJkL.png,[pipelined]
5,5eab3982888528e177caca71,https://pbs.twimg.com/media/EPnOWpwWsAQUJjx.jpg,[pipelined]
6,5eab91d31b8e1df3c9e998ba,https://pbs.twimg.com/media/ETpshmEU8AAzdd4.png,[pipelined]
7,5eababcb23288dee3ede1164,https://pbs.twimg.com/media/ETtI4xeWoAEsBYS.jpg,[pipelined]
8,5eac004cb23ffdc77614663e,https://pbs.twimg.com/media/EVpg6g8UMAUr2tz.jpg,[pipelined]
9,5eac09936eea443ef63b944b,https://pbs.twimg.com/media/EUWp2t4XYAA8PpE.jpg,[pipelined]


In [4]:
def download(folderName,_id, extension,r, MainDir):
    if extension in [".jpg", ".png"]:
        if not os.path.exists(MainDir+"/"+folderName):
            os.makedirs(MainDir+"/"+folderName)
        path = MainDir+folderName+"/"+str(_id) + extension
#         print(path)
        with open(path, 'wb') as f:
            f.write(r.content)

In [5]:
def FolderPreparator(x, MainDir):
    
    if not os.path.exists(MainDir):
        os.mkdir(MainDir)    
        
    folders = x[x==True].index.values
    _id, url, reason = x["ID"], x["url"], x["GutenTagTrash"]
    extension = url[-4:]
    try:
        r = requests.get(url, timeout=5)
        if len(reason) > 0:
            for reason_folder in reason:
    #             print(reason_folder)
                download(reason_folder,_id, extension, r, MainDir)
        else:
            download("other",_id, extension, r, MainDir)
    except Exception as err:
        pass

In [6]:
mainDir = "newData/"

In [8]:
totalDeleted = totalDeleted.progress_apply(lambda x: FolderPreparator(x, mainDir), axis=1)

100%|██████████| 2129/2129 [09:51<00:00,  3.60it/s] 


In [9]:
!zip -r alyona.zip newData

  adding: newData/ (stored 0%)
  adding: newData/pipelined/ (stored 0%)
  adding: newData/pipelined/5ed073afdb8ad633fdafa109.jpg (deflated 12%)
  adding: newData/pipelined/5ed09d0e18b726e429517f95.png (deflated 9%)
  adding: newData/pipelined/5ed24d5bdc12488afcde2da9.jpg (deflated 1%)
  adding: newData/pipelined/5ecf00665dcd536d4b5be704.png (deflated 10%)
  adding: newData/pipelined/5ed1a5da3d5a9b1d397bac02.jpg (deflated 23%)
  adding: newData/pipelined/5ed2d6599389e8515b34d162.jpg (deflated 20%)
  adding: newData/pipelined/5ed16f1d58fccbe080d7bb78.jpg (deflated 20%)
  adding: newData/pipelined/5ed1259fd2c3eec8713ab2bc.jpg (deflated 13%)
  adding: newData/pipelined/5ed1f441edcf8c29820f2790.jpg (deflated 15%)
  adding: newData/pipelined/5ecf98f24b9b77b74ec9aba6.png (deflated 0%)
  adding: newData/pipelined/5ed106245246f5dcad715e67.jpg (deflated 13%)
  adding: newData/pipelined/5ed8974a0fd52687eb5046cb.jpg (deflated 11%)
  adding: newData/pipelined/5ed272012d943591ec906d2b.png (deflated 

  adding: newData/pipelined/5ed8ac8bbc496b738bbbd3e2.jpg (deflated 19%)
  adding: newData/pipelined/5ed874f0bff1360056f48247.png (deflated 1%)
  adding: newData/pipelined/5ed7ba2f2061c3c1e24211b9.jpg (deflated 10%)
  adding: newData/pipelined/5ed9121a9cb2a9a9591c37fe.jpg (deflated 9%)
  adding: newData/pipelined/5ec836be435dae4c681025e7.png (deflated 0%)
  adding: newData/pipelined/5ed056b3ef42a8e0f6aebb67.jpg (deflated 17%)
  adding: newData/pipelined/5ec836c886707abfad8b3596.jpg (deflated 1%)
  adding: newData/pipelined/5ed1756a506cc69133e889cf.jpg (deflated 13%)
  adding: newData/pipelined/5ec836457bb5661ef8808a60.png (deflated 7%)
  adding: newData/pipelined/5ed094d8c367ba3aaf1a41e7.jpg (deflated 27%)
  adding: newData/pipelined/5eceb7a3d75af8aa3a7bceae.jpg (deflated 12%)
  adding: newData/pipelined/5ed1a4b0127f82fa3af17e3b.jpg (deflated 12%)
  adding: newData/pipelined/5ed7db1f282a36682c0fbce4.png (deflated 7%)
  adding: newData/pipelined/5ecf9c3852e9d696421a3965.png (deflated 5%)

  adding: newData/pipelined/5ed1f943d245b650ac47a645.png (deflated 3%)
  adding: newData/pipelined/5ed073f11ebd4067a7894e46.png (deflated 2%)
  adding: newData/pipelined/5ecf81f6e345db02f70dd9af.jpg (deflated 5%)
  adding: newData/pipelined/5ec8372d16b3d2ce4e5fa74f.png (deflated 4%)
  adding: newData/pipelined/5ed19c01fe62c8d38f32cff1.jpg (deflated 27%)
  adding: newData/pipelined/5ed236ff782f6ae0a4ea563f.jpg (deflated 19%)
  adding: newData/pipelined/5ed29313b1ef1774fb197000.jpg (deflated 6%)
  adding: newData/pipelined/5ed28c98ba585a3dcf970c79.png (deflated 1%)
  adding: newData/pipelined/5ed085cb876d30d6cb0ef98f.jpg (deflated 15%)
  adding: newData/pipelined/5ed90d808e7d7fdd7d65189a.png (deflated 1%)
  adding: newData/pipelined/5ed1d971cd252395c36002ef.png (deflated 5%)
  adding: newData/pipelined/5ed06b1d8e3d56e22e40e529.jpg (deflated 17%)
  adding: newData/pipelined/5ec9cccf516a065a273cc8d6.png (deflated 1%)
  adding: newData/pipelined/5ecee1a924ac792e73296381.jpg (deflated 22%)
 

  adding: newData/pipelined/5ed2919f9b60d678b3cd8bb0.jpg (deflated 19%)
  adding: newData/pipelined/5ed7c411da34aad09a453207.jpg (deflated 22%)
  adding: newData/pipelined/5eb8d48435232baa30b3165d.jpg (deflated 8%)
  adding: newData/pipelined/5ed286da45e11a5a76402bea.jpg (deflated 8%)
  adding: newData/pipelined/5ecf14438a7a1a942015298c.jpg (deflated 24%)
  adding: newData/pipelined/5ed0bba54ab3356a53d2bfc4.png (deflated 1%)
  adding: newData/pipelined/5ecf9bf7a3ad1dc44e2786fa.png (deflated 1%)
  adding: newData/pipelined/5ed04abb7bf8386cd3a59363.jpg (deflated 14%)
  adding: newData/pipelined/5ed1102eac9153abcc4f4adc.jpg (deflated 12%)
  adding: newData/pipelined/5ed37a19a7ee914ed67ba661.jpg (deflated 23%)
  adding: newData/pipelined/5ed8a964c1ebdce05e5a38c8.png (deflated 3%)
  adding: newData/pipelined/5ed072adb4ab42d9d15ff814.jpg (deflated 6%)
  adding: newData/pipelined/5ecea93ef5cd35e0932ba429.png (deflated 1%)
  adding: newData/pipelined/5ec9c9f8516a065a273cc8c4.png (deflated 3%)


  adding: newData/pipelined/5ec8305da550157c8e90ae31.png (deflated 1%)
  adding: newData/pipelined/5ec830b4195eac0a95e18c96.png (deflated 1%)
  adding: newData/pipelined/5ed90ad5b8da5454053954b5.jpg (deflated 13%)
  adding: newData/pipelined/5ed311470c8282f1345fbc07.jpg (deflated 4%)
  adding: newData/pipelined/5ed89508ad0ceaec1fcf39f0.jpg (deflated 6%)
  adding: newData/pipelined/5ed8a6a8c1ebdce05e5a38c7.jpg (deflated 20%)
  adding: newData/pipelined/5ec8360d985943a8d4779b05.png (deflated 1%)
  adding: newData/pipelined/5ed03e78823f1e7a325468a9.jpg (deflated 3%)
  adding: newData/pipelined/5ed1d85618b726e429518009.jpg (deflated 16%)
  adding: newData/pipelined/5ec82fc34d0c38481e6a3907.jpg (deflated 9%)
  adding: newData/pipelined/5eac4ced9e441b8b1dbec997.png (deflated 6%)
  adding: newData/pipelined/5ed060c444d2b388e969958e.jpg (deflated 29%)
  adding: newData/pipelined/5ed00544cfe6966d2e56d302.jpg (deflated 25%)
  adding: newData/pipelined/5ed32cf5ba585a3dcf970cbc.png (deflated 9%)
 

...

  adding: newData/pipelined/5ed2946e558dbc1fb4704e68.png (deflated 9%)
  adding: newData/pipelined/5ed20f9f42ff7b963c9dc6b5.png (deflated 3%)
  adding: newData/pipelined/5ed12dbc44bc42fb421f897a.jpg (deflated 23%)
  adding: newData/pipelined/5ed796c47e6df20d942e1716.png (deflated 2%)
  adding: newData/pipelined/5ed0bc1049e7b30f8250b8ad.jpg (deflated 9%)
  adding: newData/pipelined/5ed17234ec1e55a3a08a86a4.jpg (deflated 7%)
  adding: newData/pipelined/5ec8361dbce78b7be412f9db.jpg (deflated 1%)
  adding: newData/pipelined/5ed85d58c1483b8c3f6a31cf.png (deflated 0%)
  adding: newData/pipelined/5ec83076346715f690fc5837.jpg (deflated 19%)
  adding: newData/pipelined/5ed1b1548d4553be107ed3dc.png (deflated 3%)
  adding: newData/pipelined/5ece627a5dcd536d4b5be6d2.png (deflated 5%)
  adding: newData/pipelined/5ed2d757ad0204b65b4b9d04.jpg (deflated 6%)
  adding: newData/pipelined/5ece4a1e5dcd536d4b5be6c9.jpg (deflated 10%)
  adding: newData/pipelined/5ed835510058c4bf9a14404c.jpg (deflated 12%)
  

  adding: newData/pipelined/5ed8b67a04c57a85573cf61d.jpg (deflated 7%)
  adding: newData/pipelined/5ec8371b786e0f46b0c11fb9.jpg (deflated 15%)
  adding: newData/pipelined/5ed31164362c25a4ddd6c117.jpg (deflated 8%)
  adding: newData/pipelined/5ed79280d039731a4195e223.jpg (deflated 22%)
  adding: newData/pipelined/5ecf853f52e9d696421a395a.jpg (deflated 10%)
  adding: newData/pipelined/5ed22d519656070055fe4403.png (deflated 1%)
  adding: newData/pipelined/5ec830c52c9a10f0237d426a.jpg (deflated 11%)
  adding: newData/pipelined/5ec82fc85df470ad493c44ec.jpg (deflated 13%)
  adding: newData/pipelined/5ed32461d6acc523044b0a97.png (deflated 1%)
  adding: newData/pipelined/5ec836a9ca25d0f9a9dfaa90.jpg (deflated 7%)
  adding: newData/pipelined/5eaae294684281d51dabf5d2.jpg (stored 0%)
  adding: newData/pipelined/5ed7fee7c1ebdce05e5a3898.jpg (deflated 25%)
  adding: newData/pipelined/5ed2ee3dd6acc523044b0a84.png (deflated 7%)
  adding: newData/pipelined/5ec836ef2c9a10f0237d428b.png (deflated 3%)
  

  adding: newData/pipelined/5ed2964147a5b4ac582e3f4b.png (deflated 2%)
  adding: newData/pipelined/5ed286e0dc12488afcde2dbe.jpg (deflated 12%)
  adding: newData/pipelined/5ece6ff657bda5e06ab567fe.png (deflated 6%)
  adding: newData/pipelined/5ed8bc050e6649a82453a921.jpg (deflated 11%)
  adding: newData/pipelined/5ed81179f19e3cd95f9df72a.jpg (deflated 8%)
  adding: newData/pipelined/5ed2ee18dc12488afcde2de6.png (deflated 7%)
  adding: newData/pipelined/5ece593857bda5e06ab567f7.jpg (deflated 9%)
  adding: newData/pipelined/5ed37a21489f40409d4020cb.png (deflated 3%)
  adding: newData/pipelined/5ed131fa3c11b9067b510763.jpg (deflated 21%)
  adding: newData/pipelined/5ec830bd91c279bfb988d748.png (deflated 2%)
  adding: newData/pipelined/5ed7aef546119fd757b9df73.jpg (deflated 36%)
  adding: newData/pipelined/5ec8362b3d89132ba22662a7.png (deflated 7%)
  adding: newData/pipelined/5ec83133f4f4bc2693bf385f.jpg (deflated 35%)
  adding: newData/pipelined/5ecfc7b47af6d2915ad7c789.jpg (deflated 5%)
 

  adding: newData/pipelined/5ec8300410cf552d83526f06.jpg (deflated 8%)
  adding: newData/pipelined/5ed00d83395f2172e95aa28c.jpg (deflated 9%)
  adding: newData/pipelined/5ec8367916b3d2ce4e5fa749.png (deflated 12%)
  adding: newData/pipelined/5ed14b403c11b9067b51076e.png (deflated 1%)
  adding: newData/pipelined/5ec8372bebbe7af82c356590.png (deflated 1%)
  adding: newData/pipelined/5ece4d5a4c152bf1ea1b3694.jpg (deflated 8%)
  adding: newData/pipelined/5ed35ad86d87a41c3062fb4e.png (deflated 2%)
  adding: newData/pipelined/5ed35105964532b67e077701.png (deflated 2%)
  adding: newData/pipelined/5ed8c695ec46a705e2e86a94.png (deflated 5%)
  adding: newData/pipelined/5ecfc7bd25bad4af7a5dee2d.jpg (deflated 9%)
  adding: newData/pipelined/5ecea94a8aabf1b8a7599fa1.png (deflated 0%)
  adding: newData/pipelined/5ecf395d8aabf1b8a7599fd6.jpg (deflated 16%)
  adding: newData/pipelined/5ec836545d21f18b6987e41a.png (deflated 1%)
  adding: newData/pipelined/5ed1a315f38137a6c3a765d3.png (deflated 0%)
  ad

  adding: newData/pipelined/5ed370c4abe33a729f2e82f8.png (deflated 0%)
  adding: newData/pipelined/5ec8371016b3d2ce4e5fa74e.jpg (deflated 16%)
  adding: newData/pipelined/5ecea938da1c03cfc4371de8.png (deflated 1%)
  adding: newData/pipelined/5ecf676ccfe6966d2e56d2ce.png (deflated 1%)
  adding: newData/pipelined/5ecea9461b70471c2fc3bcaf.png (deflated 2%)
  adding: newData/pipelined/5ec836723ad936f62c626e53.png (deflated 3%)
  adding: newData/pipelined/5ed08ca1aca66ed98a4432c6.png (deflated 11%)
  adding: newData/pipelined/5ec8370311dde87f701e1038.jpg (deflated 18%)
  adding: newData/pipelined/5ecf53046ad5c1579d191f74.jpg (deflated 7%)
  adding: newData/pipelined/5ed191aa921a3da9e7f18cb2.jpg (deflated 5%)
  adding: newData/pipelined/5ed2b7b0a7ee914ed67ba60e.jpg (deflated 7%)
  adding: newData/pipelined/5ed316b16d57852008e37041.png (deflated 1%)
  adding: newData/pipelined/5ed1d13327cf08b4e586f475.png (deflated 7%)
  adding: newData/pipelined/5ed34580abdf79849b08f717.jpg (deflated 16%)
  